<a href="https://colab.research.google.com/github/aSafarpoor/SCLFB/blob/main/SCLFB_phase0_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle
import random
import json
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import cv2
from google.colab.patches import cv2_imshow
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tqdm.pandas()

#go to path

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/MyDrive/MSc/SCLAD/
%mkdir FB15k237
%cd FB15k237

/content/drive/MyDrive/MSc/SCLAD
mkdir: cannot create directory ‘FB15k237’: File exists
/content/drive/MyDrive/MSc/SCLAD/FB15k237


#read data

In [5]:
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2name.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2description.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/test.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/train.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/valid.txt

In [6]:
!ls

des.p			   FB15k_mid2name.txt  test.pkl  train.txt
FB15k_mid2description.txt  sim.pkl	       test.txt  valid.txt


In [7]:
des = []
for line in open('FB15k_mid2description.txt', 'r'):
    des.append(line.replace('\n','').split('\t'))
des = pd.DataFrame(des)
des = des.rename(columns={0: "namecode", 1: "description"})
des.head(5)

,namecode,description
0,/m/06rf7,"""Schleswig-Holstein is the northernmost of the..."
1,/m/0c94fn,"""Gary Roger Rydstrom is an American sound desi..."
2,/m/016ywr,"""Jeremy John Irons is an English actor. After ..."
3,/m/01yjl,"""The Chicago Cubs are a professional baseball ..."
4,/m/02hrh1q,"""An actor is a person portraying a character i..."


In [8]:
dfname = []
for line in open('FB15k_mid2name.txt', 'r'):
    dfname.append(line.replace('\n','').split('\t'))
dfname = pd.DataFrame(dfname)
dfname = dfname.rename(columns={0: "namecode", 1: "title"})
dfname.head(5)

,namecode,title
0,/m/06rf7,Schleswig-Holstein
1,/m/0c94fn,Gary_Rydstrom
2,/m/016ywr,Jeremy_Irons
3,/m/01yjl,Chicago_Cubs
4,/m/02hrh1q,Actor-GB


In [9]:
train = []
for line in open('train.txt', 'r'):
    train.append(line.replace('\n','').split('\t'))
train = pd.DataFrame(train)
train = train.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
train.head(5)

,namecode1,title,namecode2
0,/m/027rn,/location/country/form_of_government,/m/06cx9
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8


In [10]:
test = []
for line in open('test.txt', 'r'):
    test.append(line.replace('\n','').split('\t'))
test = pd.DataFrame(test)
test = test.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
test.head(5)

,namecode1,title,namecode2
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c
3,/m/02jx1,/location/location/contains,/m/013t85
4,/m/02jx1,/location/location/contains,/m/0m0bj


In [11]:
valid = []
for line in open('valid.txt', 'r'):
    valid.append(line.replace('\n','').split('\t'))
valid = pd.DataFrame(valid)
valid = valid.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
valid.head(5)

,namecode1,title,namecode2
0,/m/07pd_j,/film/film/genre,/m/02l7c8
1,/m/06wxw,/location/location/time_zones,/m/02fqwt
2,/m/01t94_1,/people/person/spouse_s./people/marriage/type_...,/m/04ztj
3,/m/02xcb6n,/award/award_category/winners./award/award_hon...,/m/04x4s2
4,/m/07f_7h,/film/film/release_date_s./film/film_regional_...,/m/04gzd


In [12]:
def reset_index(df):
    df = df.reset_index()
    del df['index']
    return df

test = reset_index(test)
train = reset_index(train)
valid = reset_index(valid)

#preprocessing

In [13]:
nodes = list(set(des.namecode))

In [14]:
len(nodes)

14904

In [15]:
d = {}
for i in tqdm(nodes):
    d[i]=[]

100%|██████████| 14904/14904 [00:00<00:00, 1008370.41it/s]


##train:

In [22]:
for i in tqdm(range(len(train))):
    row = train.loc[i]
    x = row[0]
    y = row[2]
    try:
        d[x].append(y)
    except:
        d[x] = [y]
        nodes.append(x)
    try:
        d[y].append(x)
    except:
        d[y] = [x]
        nodes.append(y)

100%|██████████| 272115/272115 [00:17<00:00, 15392.68it/s]


## test

In [31]:
for k in tqdm(list(d.keys())):
    d[k] = np.array(list(set(d[k])))

100%|██████████| 14929/14929 [00:00<00:00, 66160.17it/s]


In [40]:
def similarity(x,y):
    c = 0
    for i in x:
        if i in y:
            c+=1
    return c

In [41]:
x = np.array([1,2,3,4,5])
y = np.array([3,4,8])
similarity(x,y)

2

In [44]:
truecounter = 0
coldstart = 0
for i in tqdm(range(len(test))):
    row = train.loc[i]
    x = row[0]
    y = row[2]

    while(True):
        z = random.randint(0,len(des)-1)
        if  len(test[test.namecode1==x][test.namecode2==des.loc[z][0]])==0:
            try:
                s1 = similarity(d[x],d[y])
            except:
                coldstart+=1
                s1 = 0

            try:
                s2 = similarity(d[x],d[z])
            except:
                coldstart+=1
                s2 = 0
            break
    

    if s1>s2:
        truecounter += 1
    
f = len(test)-truecounter
t = truecounter
print(t,f,int(100*t/(t+f)))
    

  0%|          | 0/20466 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 20466/20466 [01:36<00:00, 212.19it/s]

16915 3551 82


test1: 82% 

test2: 82%